In [90]:
import pandas as pd

In [4]:
df = pd.read_csv(r"C:\Users\giris\Downloads\cs-training.csv\cs-training.csv")

In [5]:
df.head()

,Unnamed: 0,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,1,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
1,2,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
2,3,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
3,4,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
4,5,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0


In [6]:
df.drop('Unnamed: 0', axis=1)
df.head()

,Unnamed: 0,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,1,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
1,2,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
2,3,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
3,4,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
4,5,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0


In [7]:
df.isnull().sum()

Unnamed: 0                                  0
SeriousDlqin2yrs                            0
RevolvingUtilizationOfUnsecuredLines        0
age                                         0
NumberOfTime30-59DaysPastDueNotWorse        0
DebtRatio                                   0
MonthlyIncome                           29731
NumberOfOpenCreditLinesAndLoans             0
NumberOfTimes90DaysLate                     0
NumberRealEstateLoansOrLines                0
NumberOfTime60-89DaysPastDueNotWorse        0
NumberOfDependents                       3924
dtype: int64

In [8]:
df.shape

(150000, 12)

In [9]:
df.dropna(inplace=True)
df.shape

(120269, 12)

In [24]:
df.duplicated().value_counts()

False    120269
Name: count, dtype: int64

In [25]:
df.isnull().sum()

Unnamed: 0           0
DefaultIn2yrs        0
CreditUtilization    0
Age                  0
PastDue30_59         0
DebtToIncomeRatio    0
MonthlyIncome        0
OpenCreditLines      0
PastDue90            0
RealEstateLoans      0
PastDue60_89         0
Dependents           0
dtype: int64

In [26]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

df = df.rename(columns={
    "SeriousDlqin2yrs": "DefaultIn2yrs",
    "RevolvingUtilizationOfUnsecuredLines": "CreditUtilization",
    "age": "Age",
    "NumberOfTime30-59DaysPastDueNotWorse": "PastDue30_59",
    "DebtRatio": "DebtToIncomeRatio",
    "MonthlyIncome": "MonthlyIncome",
    "NumberOfOpenCreditLinesAndLoans": "OpenCreditLines",
    "NumberOfTimes90DaysLate": "PastDue90",
    "NumberRealEstateLoansOrLines": "RealEstateLoans",
    "NumberOfTime60-89DaysPastDueNotWorse": "PastDue60_89",
    "NumberOfDependents": "Dependents"
})


In [27]:
df_majority = df[df["DefaultIn2yrs"] == 0]
df_minority = df[df["DefaultIn2yrs"] == 1]

print("Before balancing:")
print(df["DefaultIn2yrs"].value_counts())

df_majority_downsampled = df_majority.sample(n=len(df_minority), random_state=42)

df_balanced = pd.concat([df_majority_downsampled, df_minority])

df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

print("\nAfter balancing:")
print(df_balanced["DefaultIn2yrs"].value_counts())


Before balancing:
DefaultIn2yrs
0    111912
1      8357
Name: count, dtype: int64

After balancing:
DefaultIn2yrs
0    8357
1    8357
Name: count, dtype: int64


In [28]:
target = "DefaultIn2yrs"
features = [
    "CreditUtilization", "Age", "PastDue30_59", "PastDue60_89", "PastDue90",
    "DebtToIncomeRatio", "MonthlyIncome", "OpenCreditLines", "RealEstateLoans", "Dependents"
]

X = df_balanced[features]
y = df_balanced[target]

In [29]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split( X_scaled, y, test_size=0.2, random_state=42)

print("Training set size:", X_train.shape)
print("Test set size:", X_test.shape)

Training set size: (13371, 10)
Test set size: (3343, 10)


In [30]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier

In [31]:
clf1 = LogisticRegression()
clf2 = KNeighborsClassifier()
clf3 = RandomForestClassifier()
clf4 = GradientBoostingClassifier()
clf5 = XGBClassifier()

In [32]:
estimators = [('lr', clf1), ('knn', clf2), ('rf', clf3), ('gbc', clf4), ('xgb', clf5)]

In [33]:
import numpy as np
for estimator in estimators:
    x = cross_val_score(estimator[1], X_scaled,y,cv=10,scoring='accuracy')
    print(estimator[0], np.round(np.mean(x),2))

lr 0.73
knn 0.64
rf 0.77
gbc 0.77
xgb 0.76


In [80]:
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.ensemble import GradientBoostingClassifier

gb_model = GradientBoostingClassifier(random_state=42)

param_grid_gb = {
    "n_estimators": [100, 200, 300],
    "learning_rate": [0.01, 0.05, 0.1],
    "max_depth": [3, 5, 7],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 3, 5]
}

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)


grid_gb = GridSearchCV(
    estimator=gb_model,
    param_grid=param_grid_gb,
    scoring="f1",        
    n_jobs=-1,
    cv=cv,
    verbose=2
)


grid_gb.fit(X_train, y_train)

print("Best Parameters (GBM):", grid_gb.best_params_)
print("Best CV Score (GBM):", grid_gb.best_score_)


Fitting 5 folds for each of 243 candidates, totalling 1215 fits
Best Parameters (GBM): {'learning_rate': 0.05, 'max_depth': 5, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 100}
Best CV Score (GBM): 0.7739200746198808


In [81]:
from xgboost import XGBClassifier

xgb_model = XGBClassifier(
    objective="binary:logistic", 
    eval_metric="logloss", 
    use_label_encoder=False,
    random_state=42
)

param_grid_xgb = {
    "n_estimators": [100, 200, 300],
    "learning_rate": [0.01, 0.05, 0.1],
    "max_depth": [3, 5, 7],
    "subsample": [0.7, 0.8, 1.0],
    "colsample_bytree": [0.7, 0.8, 1.0],
    "gamma": [0, 0.1, 0.3]
}


grid_xgb = GridSearchCV(
    estimator=xgb_model,
    param_grid=param_grid_xgb,
    scoring="f1",      
    n_jobs=-1,
    cv=cv,
    verbose=2
)


grid_xgb.fit(X_train, y_train)

print("Best Parameters (XGB):", grid_xgb.best_params_)
print("Best CV Score (XGB):", grid_xgb.best_score_)


Fitting 5 folds for each of 729 candidates, totalling 3645 fits
Best Parameters (XGB): {'colsample_bytree': 0.8, 'gamma': 0.3, 'learning_rate': 0.05, 'max_depth': 5, 'n_estimators': 100, 'subsample': 1.0}
Best CV Score (XGB): 0.7743129426486559


C:\Users\giris\anaconda3\envs\torch_env\lib\site-packages\xgboost\core.py:158: UserWarning: [12:45:09] WARNING: D:\bld\xgboost-split_1744329000128\work\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


In [82]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold

rf = RandomForestClassifier(random_state=42)

param_grid_rf = {
    "n_estimators": [100, 200, 300],   
    "max_depth": [None, 5, 10, 15],    
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
    "max_features": ["sqrt", "log2"]   
}

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

grid_rf = GridSearchCV(
    estimator=rf,
    param_grid=param_grid_rf,
    scoring="f1",   
    n_jobs=-1,
    cv=cv,
    verbose=2
)

grid_rf.fit(X_train, y_train)

print("Best Params RF:", grid_rf.best_params_)
print("Best CV Score RF:", grid_rf.best_score_)


Fitting 5 folds for each of 216 candidates, totalling 1080 fits
Best Params RF: {'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Best CV Score RF: 0.7737537247155356


In [83]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, classification_report

def compare_models(models, X_test, y_test):
    """
    Compare multiple trained models on Accuracy, F1, and ROC-AUC.
    
    models: dict of { "ModelName": trained_model }
    """
    results = []

    for name, model in models.items():
        
        y_pred = model.predict(X_test)
        y_pred_proba = model.predict_proba(X_test)[:, 1]

        acc = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        roc = roc_auc_score(y_test, y_pred_proba)

        results.append([name, acc, f1, roc])

        print(f"\n{name} Results")
        print("-" * 30)
        print("Accuracy:", round(acc, 3))
        print("F1-score:", round(f1, 3))
        print("ROC-AUC:", round(roc, 3))
        print("\nClassification Report:")
        print(classification_report(y_test, y_pred))


    import pandas as pd
    df_results = pd.DataFrame(results, columns=["Model", "Accuracy", "F1", "ROC-AUC"])
    print("\nSummary Table:")
    print(df_results)
    return df_results


In [84]:
best_rf = grid_rf.best_estimator_
best_gb = grid_gb.best_estimator_
best_xgb = grid_xgb.best_estimator_

models = {
    "Random Forest": best_rf,
    "Gradient Boosting": best_gb,
    "XGBoost": best_xgb
}

results_df = compare_models(models, X_test, y_test)


Random Forest Results
------------------------------
Accuracy: 0.768
F1-score: 0.759
ROC-AUC: 0.846

Classification Report:
              precision    recall  f1-score   support

           0       0.75      0.80      0.78      1684
           1       0.78      0.74      0.76      1659

    accuracy                           0.77      3343
   macro avg       0.77      0.77      0.77      3343
weighted avg       0.77      0.77      0.77      3343


Gradient Boosting Results
------------------------------
Accuracy: 0.765
F1-score: 0.758
ROC-AUC: 0.847

Classification Report:
              precision    recall  f1-score   support

           0       0.76      0.79      0.77      1684
           1       0.77      0.74      0.76      1659

    accuracy                           0.76      3343
   macro avg       0.77      0.76      0.76      3343
weighted avg       0.76      0.76      0.76      3343


XGBoost Results
------------------------------
Accuracy: 0.765
F1-score: 0.759
ROC-AUC: 0.8

In [85]:
from sklearn.metrics import classification_report, precision_recall_curve
import numpy as np

y_pred_proba = best_xgb.predict_proba(X_test)[:, 1]

thresholds = np.arange(0.2, 0.6, 0.02)  # test from 0.2 to 0.6
for t in thresholds:
    y_pred_custom = (y_pred_proba >= t).astype(int)
    report = classification_report(y_test, y_pred_custom, output_dict=True)
    recall_class1 = report['1']['recall']
    if recall_class1 >= 0.80:
        print(f"✅ Threshold {t:.2f} gives recall={recall_class1:.2f}, "
              f"precision={report['1']['precision']:.2f}, "
              f"f1={report['1']['f1-score']:.2f}")
        break

✅ Threshold 0.20 gives recall=0.94, precision=0.62, f1=0.75


In [87]:
chosen_threshold = 0.40  

y_pred_custom = (y_pred_proba >= chosen_threshold).astype(int)

from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_custom))


              precision    recall  f1-score   support

           0       0.80      0.70      0.75      1684
           1       0.73      0.82      0.77      1659

    accuracy                           0.76      3343
   macro avg       0.77      0.76      0.76      3343
weighted avg       0.77      0.76      0.76      3343



In [89]:
def predict_with_threshold(model, X, threshold=0.40):
    proba = model.predict_proba(X)[:, 1]
    return (proba >= threshold).astype(int)
